---
<div align="justify">
    <h1><strong>An automation approach of the mechanical design and manufacturing of ball mills</strong></h1>
    <h2>Juan D. Argüello$^a$, Omar A. Gélvez$^b$, Jairo R. Martínez$^c$, Diego C. Durán$^c$, Elena E. Stashenko$^c$</h2>
    <foot><i>$^a$Universidad Industrial de Santander, Escuela de Ingeniería Mecánica, Centro Nacional de Investigaciones para la Agroindustrialización de Especies Vegetales Aromáticas y Medicinales Tropicales - CENIVAM. Bucaramanga, Colombia.</i></foot>
    <br>
    <foot><i>$^b$Universidad Industrial de Santander, Escuela de Ingeniería Mecánica, Grupo de Investigación en Energía y Medio Ambiente - GIEMA. Bucaramanga, Colombia.</i></foot>
    <br>
    <foot><i>$^c$Universidad Industrial de Santander, Escuela de Química, Centro de Investigación en Biomoléculas - CIBIMOL y Centro Nacional de Investigaciones para la Agroindustrialización de Especies Vegetales Aromáticas y Medicinales Tropicales - CENIVAM. Bucaramanga, Colombia.</i></foot>
</div>

---

<div align="center">
    <h2><strong>Abstract</strong></h2>
    Engineering design is taking a new step forward with technological advances. <i>Open source</i> tools, such as Python, make it possible to develop calculation and validation algorithms to automate design procedures. Ball mills stands out as a solution for crushing processes, specially in the civil industry, which its design procedure requires the study of the cynematic of the balls and the dynamical interaction between them and the mill. Numerical simulations are usually used as a validation tool, but also as an optimization one. For this particular case, there were used two numerical methods: the <i>Discrete Element Method</i> (DEM), used to validate the cynematic theoretical study, optimize it and to predict the impact distribution energy over the mill; and the <i>Finite Element Method</i> (FEM), which was used to predict, and guarantee, an adecuate structural behaviour of the mill. The present work also create automatic engineering reports (both in TeX and PDF formats), CAD drawings of the designed ball mill (dxf format) and an automatic quantification of the required materials for manufacturing. All of the above can be summarized in three principal stages, or chapters: the first one is related to the theoretical study, the second one to the numerical simulations and the last one to the obtained results.
</div>

<br>

_Keywords:_ ball mill, automation, mechanical design, FEM, DEM, FEniCS, Python.

---

## __1. Introduction__

### _1.1. Background and Motivation_

<div align="justify">
    In recent years, there have been reported several studies where automation has been used in engineering design. For the production of nuclear energy, it can be found different studies, such as: the design of reactor cores$^{[1]}$, from where Hyun Kim <i>et al</i> uses an artificial neural network to provide, and evaluate, new types of geometries; Nomen <i>et al</i> developed a numerical methodology to create the design of IFMIF LIPAc beam dump shielding$^{[2]}$, which provides an acelerator-based, D-Li neutron source to produce energy of high intensity by radiation; Kim J. <i>et al</i> created an autonomous operation system$^{[3]}$ with artificial intelligence which perform the control functions needed for the emergency operation of a nuclear power plant.
    <br>
    <br>
    In civil engineering, there is a specialized journal known as "Automation in Construction", which has published different studies related to automatic structural and seismic designs. In mechanical design, it can be found articles related with automation in different industries, like automotive, fluid transport and robotics, to name a few. For example: Ouyang T. <i>et al</i> developed a dynamic modelling of a clutch actuator for heavy duty transmission$^{[4]}$ adopting an artificial bee colony (ABC) algorithm to optimize structural parameters; X. Telleria <i>et al</i> presented a methodology to automate the design, and numerical validation, of valves$^{[5]}$; M. Honarpardaz <i>et al</i> introduced the Generic Automated Finger Design$^{[6, 7]}$ (GAFD) method for design automation of customized fingers of industrial grippers for robots. 
    <br>
    <br>
   Most of these works are intended to study a particular stage of the design, either modelling, numerical method or simulation. The motivation of doing this work was to propose a design methodology, which groups all of this efforts, focusing in the required information for the manufacturing of the mill.
</div>


### _1.2. Future Updates_

<div align="justify">
    Things can always be done better or differently. The main idea of this work was to prove a design methodology, but the possibility to do things "simpler" from the user perspective is still there. To achieve this, an artificial intelligence algorithm must be implemented. 
    <br>
    <br>
    Another future update should let advanced users to implement experimental grinding equations to predict time consuming.
</div>

## __2. Theoretical study__

<div align="justify">
    The goal of this section is to find:
</div>

* Power.
* Diameter of the mill.
* Length of the mill.
* Rotational speed.
* Number of balls.

### _2.1. Requirements and Specifications_

<div align="justify">
    The <i>default</i> data can be apreciated next.
</div>

#### _2.1.1. Requirements_

* Capacity of $20 \left[kg/batch \right]$.
* Input particle size: $850 \left[ \mu m \right]$.
* Target particle size: $250 \left[ \mu m\right]$.

#### _2.1.2. Specifications_

* Length - diameter relation of 1.5 to 3.
* Ball material: Aluminum $\left(\rho _{Al} = 2700 \left[kg/m^3 \right] \right)$.
* Grinding material: Granite $\left(\rho _{g} = 2520 \left[kg/m^3 \right] \right)$.
* Occupied volume by the material between 40% and 60%.
* Occupied volume by the balls between 10% and 30%.
* Rotational speed should be the 80% of the critical velocity.
* Safety factor between 2 and 4.


In [1]:
from App.Data import ReqEs
data = ReqEs.Datos()
data

### _2.2. Mill volume $\rightarrow$ Diameter and Length_

<div align="justify">
    The mill volume, shown on Figure 1, is directly proportional to the sum of the volumes occupied by the grinding material and the balls. It is indirectly proportional to the volume ratio of <i>all</i> of the material. 
    <div align="center">
        <img src="Images/Preprocesamiento/volumen.PNG" style="width: 500px;" />
    </div>
    <div align="center">
        <i>Figura 1.</i> Relation between mass and volume.
    </div>
</div>

<div align="justify">
    $$
    \begin{equation}
        V = \frac{V_{Gm} + V_{Tb}}{P}
        \tag{1}
        \label{VT}
    \end{equation}
    $$
    From Equation \ref{VT}, $V$ is the volume of the mill, $V_{Gm}$ is the volume occupied by the grinding material, $V_{Tb}$ correspond to the total volume occupied by the balls and $P$ is the percentage of the mill volume occupied by the load.
</div>

In [6]:
from IPython.display import HTML, display
import pandas as pd
from App.Data.Read import Read
from App.Funcional.Dimensiones import *
datos = Read(data)
res = {'Vol':{}}
res['Vol'], vol = Volumenes(Read(data))()

volumenes = pd.DataFrame(vol)
display(HTML("The <strong>volume of the mill</strong> is $" + 
             str(vol['Valor $[L]$'][2]) + " [L]$"))
volumenes

,Tipo,Valor $[L]$
0,$V_{Gm}$,166.67
1,$V_{Tb}$,41.67
2,$V_{T}$,416.67


<div align="justify">
    Knowing the volume of the mill, the length and diameter can be calculated as follows:
    $$
    \begin{equation}
        V = \frac{\pi}{4} D^2 L
        \tag{2}
        \label{volCil}
    \end{equation}
    $$
</div>

In [9]:
res['D'], res['L'] = dim(res['Vol']['V'], datos['Esp']['Length - diameter relation'])
display(HTML("The diameter $D$ has a value of: $" + str(round(res['D'],2)) + 
             " [m]$ and the length of the mill $L$ is: $" + str(round(res['L'],2)) + " [m]$"))

### _2.3. Rotational speed of the mill_

<div align="justify">
    The analysis to define the optimum rotational speed is based on the principle that the minimum rotational speed should made the balls adhered to the surface of the mill.
</div>

<div align="center">
        <img src="Images/Preprocesamiento/DBola.PNG" style="width: 400px;" />
</div>
<div align="center">
    <i>Figura 2.</i> Dynamic of a ball.
</div>

<div align="justify">
    Sum of forces in the radial direction gives:
    $$
    \begin{equation}
        \sum F_r = 0 = C - W cos \, \alpha \\
        \rightarrow \boxed{C = W cos \, \alpha}
        \tag{3}
        \label{Fr}
    \end{equation}
    $$
    The dynamic of the ball is defined by:
    $$
    \begin{equation}
        \vec{C} = m \vec{a_p} 
        \tag{4}
        \label{dinB}
    \end{equation}
    $$
    Where: $m$ is the mass of the ball and $\vec{a_p}$ is the acceleration vector. The absolute acceleration is equivalent to the sum of three accelerations: rotational acceleration, relative acceleration and the Coriolis acceleration.
</div>

<div align="justify">
    $$
    \begin{equation}
        \vec{a_p} = \dot{\vec{V_p}} = \dot{\vec{\omega}} \times \vec{r} + \vec{\omega} \times \left(\vec{\omega} \times \vec{r} \right) + 2 \vec{\omega} \times \left(\dot{\vec{r}} \right) _{rot} + \left(\ddot{\vec{r}} \right) _{f}
        \tag{5}
        \label{apG}
    \end{equation}
    $$
    The rotational speed of the mill is constant, which implies that the acceleration of the particle behaves as shown on Equation \ref{ap}.
    $$
    \begin{equation}
        \vec{a_p} = \vec{\omega} \times \left(\vec{\omega} \times \vec{r} \right)
        \tag{6}
        \label{ap}
    \end{equation}
    $$
    Relating Equations \ref{Fr}, \ref{dinB} and \ref{ap}:
    $$
    \begin{equation}
        C = W cos \, \alpha = m \left(\omega ^2 r \right) = \left(\frac{W}{g} \right) \left[ \left(\frac{V}{r} \right) ^2 r \right]  \rightarrow \\
        W cos \, \alpha = \frac{W \left(2\pi r N(\alpha) \right)^2}{rg} \rightarrow \boxed{N(\alpha) = \sqrt{\frac{g cos \, \alpha}{4 \pi ^2 r}}}
        \tag{7}
        \label{Nalpha}
    \end{equation}
    $$
    Comparing Equation \ref{Nalpha} with Figure 2, it can be seen that the maximum value of the velocity that the mill should rotate is reached when $\alpha = 0$.
    $$
    \begin{equation}
        N_{max} = \sqrt{\frac{g}{4 \pi ^2 r}}
        \tag{8}
        \label{N}
    \end{equation}
    $$
</div>

In [13]:
import math
general = {'g':{'valor':9.81, 'units':'m/s2'}}
N_max = (general['g']['valor']/(4*math.pi**2*(res['D']/2)))**(1/2)
res['N'] = N_max*(datos['Esp']\
                  ['Rotational speed criteria [%]']/100)*60
display(HTML("The critical rotational speed $N_{máx}$ is of $" + str(round(N_max,3)) +
            " [rev/s] \\rightarrow" + str(round(N_max*60,3)) + " [rpm]$. "
             "Applying the <i>rotational speed criteria</i> ($ " +
             str(datos['Esp']['Rotational speed criteria [%]']) +
             " \% $ of the maximum speed)" +
            " the rotational speed $N$ of the mill is: $" + 
             str(round(res['N'],2)) +
            " [rpm]$."))

### _2.4. Number of balls_

<div align="justify">
    The user can specify the diameter of the balls, and how many types there shloud be. The number of balls is calculated by the Equation \ref{Nb}. It has, by default, two types of balls: $2 [cm]$ and $4 [cm]$.
    $$
    \begin{equation}
        N_b = \frac{V_{Tb}}{\frac{4}{3}\pi \left(\frac{D_b}{2} \right) ^2}
        \tag{9}
        \label{Nb}
    \end{equation}
    $$
</div>

In [63]:
from ipywidgets import *

def BallTypes(NumTypes, default=True):
    child = []
    if default:
        vals = [2,4]
        for i in range(2):
            child.append(FloatText(value=vals[i]))
    else:
        for i in range(NumTypes):
            child.append(FloatText(value=2))
    Ac = Accordion(children=child)
    for i in range(len(child)):
        Ac.set_title(i, 'Ball ' + str(i+1) + ' [cm]')
    display(Ac)
    return Ac, NumTypes
            
BT = interactive(BallTypes,
                 default=True,
                 NumTypes = IntText(value=2))
display(BT)

interactive(children=(IntText(value=2, description='NumTypes'), Checkbox(value=True, description='default'), O…

In [78]:
from math import floor
from IPython.display import Markdown
TB = {}
for i in range(len(BT.result[0].children)):
    TB[BT.result[0].children[i].value] = 0
suma = 0
for tipo in TB:
    suma += 1
V = (res['Vol']['V_Tb']/suma,)
display(HTML("Number of balls per type:"))
for tipo in TB:
    Vb = (4/3)*math.pi*(tipo/(2*100))**2
    res['NBolas'][str(tipo)] = {}
    res['NBolas'][str(tipo)]['Cant'] = floor(V[0]/Vb)
    res['NBolas'][str(tipo)]['masa'] = res['NBolas'][str(tipo)]['Cant']*\
        datos['Esp']['Density of balls [kg/m3]']*Vb
    display(Markdown("* $" + str(tipo) + "[cm]$: " + str(res['NBolas'][str(tipo)]['Cant'])))

HTML(value='Number of balls per type:')

* $2.0[cm]$: 49

* $4.0[cm]$: 12

### _2.5. Power_

<div align="justify">
    Para el cálculo de la potencia mínima que garantice el funcionamiento del sistema de preprocesamiento, se desarrolla el análisis dinámico mostrado en la Figura 5.
</div>

<div align="center">
    <img src="Images/Preprocesamiento/CM.PNG" style="width: 500px;" />
</div>
<div align="center">
    <i>Figura 5.</i> Diagrama de cuerpo libre del molino.
</div>

<div align="justify">
    Conociendo la velocidad de rotación, se puede calcular la potencia como:
    $$
    \begin{equation}
        Pot = T \omega \\
        T = F.S \, W\, d\, sin\left(\alpha \right) 
        \tag{10}
        \label{pot}
    \end{equation}
    $$
    Dónde: $F.S$ es el factor de seguridad, $W$ es la carga del molino y $d$ es la distancia al centro de masa.
</div>

#### 3.6.1. Distancia al centro de masa

<div align="justify">
    La distancia al centro de masa, se evalúa de la siguiente forma:
    $$
    \begin{equation}
        d = \sqrt{\overline{x}^2 + \overline{y}^2} \\
        \overline{x} A = \int x \, dA \\
        \overline{y} A = \int y \, dA
        \tag{11}
        \label{dcm}
    \end{equation}
    $$
    El área de una región circular se calcula como:
    $$
    \begin{equation}
        A = \frac{1}{2} \theta r^2
        \tag{12}
    \end{equation}
    $$
    El ángulo $\alpha$ que permite tener la mayor energía de impacto es de $45°$. Para convertir la Ecuación \ref{dcm} en coordenadas polares:
    $$
    \begin{equation}
        x = r cos \, \theta \\
        y = r sin \, \theta \\
        dA = r \, dr \, d\theta
    \end{equation}
    $$
</div>

In [85]:
from sympy import *
#Datos...
r, theta = symbols('r, \\theta', real=True, positive=True)
A, xbar, ybar,d = symbols("A, \\bar{x}, \\bar{y},d")
dA = r
x = r*cos(theta)
y = r*sin(theta)

#Solución
AA = integrate(dA, (r, 0, r), (theta,-pi/2,pi/2))
xxbar = integrate(x*dA, (r, -r,0), (theta,(5/4)*pi,pi/4))/AA
yybar = integrate(y*dA, (r, -r,0), (theta,(5/4)*pi,pi/4))/AA
dd = sqrt(xxbar**2 + yybar**2)

#Ecuaciones
Area = Eq(A,Integral(dA, (r, -r, 0), (theta,(5/4)*pi,pi/4)))
display(Markdown("1. Área:"))
display(Area)
display(Eq(A, AA))

1. Área:

Eq(A, Integral(r, (r, -r, 0), (\theta, 1.25*pi, pi/4)))

Eq(A, pi*r**2/2)

In [86]:
xx = Eq(xbar, (1/A)*Integral(x*dA, (r, -r,0), (theta,(5/4)*pi,pi/4)))
display(Markdown("2. $\\bar{x}$:"))
display(xx)
display(Eq(xbar, xxbar))

2. $\bar{x}$:

Eq(\bar{x}, Integral(r**2*cos(\theta), (r, -r, 0), (\theta, 1.25*pi, pi/4))/A)

Eq(\bar{x}, 2*sqrt(2)*r/(3*pi))

In [87]:
yy = Eq(ybar, (1/A)*Integral(y*dA, (r, -r,0), (theta,(5/4)*pi,pi/4)))
display(Markdown("3. $\\bar{y}$:"))
display(yy)
display(Eq(ybar, yybar))
#Solución
yybar = integrate(y*dA, (r, -r,0), (theta,(5/4)*pi,pi/4))/AA
dd = sqrt(xxbar**2 + yybar**2)
display(Markdown("4. distancia:"))
display(Eq(d, sqrt(xbar**2 + ybar**2)))
display(Eq(d, dd))

3. $\bar{y}$:

Eq(\bar{y}, Integral(r**2*sin(\theta), (r, -r, 0), (\theta, 1.25*pi, pi/4))/A)

Eq(\bar{y}, -2*sqrt(2)*r/(3*pi))

4. distancia:

Eq(d, sqrt(\bar{x}**2 + \bar{y}**2))

Eq(d, 4*r/(3*pi))

#### 3.6.2. Cálculo de la potencia

In [9]:
d = dd.subs({r:res['D']/2}).evalf()
W = datos['Req']['Capacidad [kg/bache]']*(1+datos['Req']['Relación másica AD - MV'])
for key in res['NBolas']:
    W += res['NBolas'][key]['masa']
W = W*9.81
T = datos['Esp']['Factor de seguridad']*W*d*math.sin(math.radians(45))
w = res['N']*(2*math.pi/60)
res['Pot'] = T*w*(1.34102/1000)
display(HTML("La <strong>potencia teórica</strong> es de $ " + str(round(res['Pot'],2)) + " [hp]$"))

In [ ]:
W

### 3.7. Resumen de resultados

<div align="justify">
    Los resultados obtenidos se pueden apreciar a continuación:
</div>

In [10]:
from App.Funcional.Resultados import *
result = Ress(res)
pd.DataFrame(result)

,Resultados
Diámetro [m],0.954
Longitud [m],1.431
Potencia [hp],6.087
Velocidad de rotación [rpm],34.648
Volumen [m3],1.022
